In [3]:
import csv
import numpy as np
import scipy as sp
import matplotlib as mpl
import math
import operator
!ls collaboration_scoring*.csv

collaboration_scoring_scenario1_single.csv   collaboration_scoring_scenarios.csv
collaboration_scoring_scenario3_overtalk.csv


In [3]:
with open("collaboration_scoring_scenario3_overtalk.csv") as baseFile:
    csvr = csv.reader(baseFile)
    records=[]
    for line in csvr:
        records.append(line)

In [59]:
print(records[0])
print(records[1])
print(records[2])
#filter(lambda x:(operator.gt(int(x[3]),0)),records[1:])

['time', 'hangout_id', 'broadcaster', 'volume']
['1436304883834', '1', 'hangout63F36BBE_ephemeral.id.google.com^a8a848dd20', '0']
['1436304883834', '1', 'hangoutB1237962_ephemeral.id.google.com^c49dc482c0', '0']


In [46]:
#Testing the data set as a whole. This gets some basic sanity checks done.
get = lambda n: ([int(k[n]) for k in records[1:]])
times = get(0)
hangout_ids = set(get(1))
participant_ids = set([k[2] for k in records[1:]])#Not an int.
vols = get(3)
vol_range = set(vols)

In [35]:
import matplotlib.pyplot as plt

plt.figure("Sanity Check")
plt.subplot(211)
plt.plot(times)
plt.subplot(212)
plt.plot(vols)
plt.figure(2)
plt.subplot(211)
plt.plot(times,vols)
plt.subplot(212)

histo_getter = lambda n,l: len([m for m in l if m==n])
lnhg = lambda n,l: (math.log(histo_getter(n,l)))
vg = lambda n:lnhg(n,vols)
vol_histogram = [vg(0),vg(1),vg(2),vg(3),vg(4),vg(5)]
plt.title("Natural log of frequencies of different volumes.")
plt.plot([0,1,2,3,4,5],vol_histogram)

In [37]:
#plt.show()

In [71]:
#Now we can compare between participants.

def by_participant(records):
    participant_ids = sorted(set([k[2] for k in records[1:]]))
    tot = []
    for participant in participant_ids:
        tot.append([record for record in records if record[2]==participant])
    return tot
bpr = by_participant(records)
#This is a list, for each participant
print(len(records))
print(len(bpr))
print(len(bpr[0])+len(bpr[1])+len(bpr[2])+len(bpr[3]))

1266
4
1265


In [39]:
#Now we compare between hangouts.
def by_participant(records):
    hangout_ids = set([int(k[1]) for k in records[1:]])
    tot = []
    for hangout in hangout_ids:
        tot.append([record for record in records if record[1]==hangout])
    return tot

In [40]:
def scrub_nulls(min_volume,records):
    return [record for record in records if record[3]>=min_volume]

In [81]:
x = filter(lambda x:(operator.gt(int(x[3]),1)),records[1:])
diffs = [[int(x[n][0])-int(x[n-1][0]), k[1],k[2],k[3]] for n,k in enumerate(x)][1:]
len(by_participant(x)[0])
len(by_participant(x)[1])
counts = [len(k) for k in by_participant(x)]
#TODO: Convert this into a REALLY BASIC collaboration score that doesn't penalize larger groups?
fractions = [float(count)/sum(counts) for count in counts]
HIndex = sum(map(lambda x:(x**2),fractions))
HIndex

0